# Adversarial Regularization for 1D CNN Toy Model

#### Adapted from tensorflow Adversarial Regularization for Image Classification tutorial:

https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist

[github](https://github.com/tensorflow/neural-structured-learning/blob/master/g3doc/tutorials/adversarial_keras_cnn_mnist.ipynb)



## Setup

Install the Neural Structured Learning package.

In [ ]:
#!pip install --quiet neural-structured-learning

In [ ]:
import matplotlib.pyplot as plt
import neural_structured_learning as nsl
import numpy as np
import tensorflow as tf

plt.rcParams['figure.dpi'] = 90

In [ ]:
print('TensorFlow Version {}'.format(tf.__version__))

## Hyperparameters


Hyperparamaters explanations from https://www.tensorflow.org/neural_structured_learning/tutorials/adversarial_keras_cnn_mnist


Input/Output:

*   **`input_shape`**: The shape of the input tensor. Each image is 28-by-28
pixels with 1 channel.
*   **`num_classes`**: There are a total of 10 classes, corresponding to 10
digits [0-9].

Model architecture:

*   **`conv_filters`**: A list of numbers, each specifying the number of
filters in a convolutional layer.
*   **`kernel_size`**: The size of 2D convolution window, shared by all
convolutional layers.
*   **`pool_size`**: Factors to downscale the image in each max-pooling layer.
*   **`num_fc_units`**: The number of units (i.e., width) of each
fully-connected layer.

Training and evaluation:

*  **`batch_size`**: Batch size used for training and evaluation.
*  **`epochs`**: The number of training epochs.

Adversarial learning:

*   **`adv_multiplier`**: The weight of adversarial loss in the training
objective, relative to the labeled loss.
*   **`adv_step_size`**: The magnitude of adversarial perturbation.
*  **`adv_grad_norm`**: The norm to measure the magnitude of adversarial
perturbation.


In [ ]:
# this notebook does note use any hyperparameters to make the base model

class HParams(object):
    def __init__(self):
#         self.input_shape = [10, 1]
#         self.num_classes = 10
#         self.conv_filters = [64, 3]
#         self.kernel_size = (3)
#         self.pool_size = (2)
#         self.num_fc_units = [10]
#         self.batch_size = 20
        self.epochs = 10
        self.adv_multiplier = 1.0
        self.adv_step_size = 2
        self.adv_grad_norm = 'infinity'

HPARAMS = HParams()

## Functions

In [ ]:
def getLabels(array):
    return np.array(array[-1], dtype=float)

In [ ]:


# FEATURE_INPUT_NAME = ['pt', 'eta', 'phi', 'mass', 'thetalab', 'radiilab', 'zlab']
FEATURE_INPUT_NAME = ['pt', 'thetalab']
LABEL_INPUT_NAME = 'labels'

## Load Perturbed and Un-perturbed samples

npz files are output of `UBCMS-XAI/AdversarialNetworks/ToyModel/makeFourVectors.ipynb`

In [ ]:
# perturbed samples
perturbed_data_train = np.load('data/jetConstTrain_overlap_perturb.npz')
perturbed_data_test = np.load('data/jetConstTest_overlap_perturb.npz')
perturbed_batch = len(perturbed_data_train['pt'])
perturbed_feat_all = [key for key in perturbed_data_train.keys()]
perturbed_feat_all.remove('labels')
print('Perturbed dataset features:')
print(perturbed_feat_all)
print()

In [ ]:
# un-perturbed samples
data_train = np.load('data/jetConstTrain_overlap.npz')
data_test = np.load('data/jetConstTest_overlap.npz')
batch = len(data_train['pt'])
feat_all = [key for key in data_train.keys()]
feat_all.remove('labels')
print('Un-perturbed dataset features:')
print(feat_all)
print()

## Create perturbed dataset dictionaries and lists

In [ ]:
perturbed_data_train_dict = {}
perturbed_data_test_dict  = {}

for key in perturbed_data_train.keys():
    if not 'labels' in key:
        perturbed_data_train_dict[key] = perturbed_data_train[key].reshape(20000, 10, 1)
        perturbed_data_test_dict[key] = perturbed_data_test[key].reshape(20000, 10, 1)

perturbed_data_train_dict['labels'] = perturbed_data_train['labels'][:,1]
perturbed_data_test_dict['labels'] = perturbed_data_test['labels'][:,1]


perturbed_data_train_list = [perturbed_data_train_dict[key] for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]]
perturbed_data_test_list  = [perturbed_data_test_dict[key]  for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]]



## Create un-perturbed dataset dictionaries and lists

In [ ]:
data_train_dict = {}
data_test_dict  = {}

for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]:
    if not 'labels' in key:
        data_train_dict[key] = data_train[key].reshape(20000, 10, 1)
        data_test_dict[key] = data_test[key].reshape(20000, 10, 1)

data_train_dict['labels'] = data_train['labels'][:,1]
data_test_dict['labels'] = data_test['labels'][:,1]
        

data_train_dict_subset = {key: data_train_dict[key] for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]}
data_test_dict_subset  = {key: data_test_dict[key]  for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]}

data_train_list = [data_train_dict[key] for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]]
data_test_list  = [data_test_dict[key]  for key in FEATURE_INPUT_NAME+[LABEL_INPUT_NAME]]



In [ ]:
sig_true = (getLabels(data_train_list) == 1)
bkg_true = (getLabels(data_train_list) == 0)
sig_pert = (getLabels(perturbed_data_train_list) == 1)

for i, (feature, perturbed_feature) in enumerate(zip(data_train_list, perturbed_data_train_list)):
    
    if(i == len(perturbed_data_train_list) - 1): continue

    bmax = np.max(feature.flatten())
    bmin = np.min(feature.flatten())
    
    plt.hist(feature[sig_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5, label='Signal')
    plt.hist(feature[bkg_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5, label='Background')
    plt.hist(perturbed_feature[sig_pert].flatten(), bins=np.linspace(bmin, bmax, 50), color='k', histtype='step', label='Perturbed Signal')
    plt.xlabel(FEATURE_INPUT_NAME[i])
    plt.ylabel('Counts')
    plt.legend()
    plt.show()

## Base model


In [ ]:
def build_base_model():


    input_shape = (10, 1)
    inputs = [tf.keras.Input(shape=input_shape, dtype=tf.float32, name=name) for name in FEATURE_INPUT_NAME]
    print(inputs)
    
    x = tf.keras.layers.concatenate(inputs = [*inputs], axis=-1, name = 'concat')
    
    print(x)
    
    x = tf.keras.layers.Conv1D(64, 3, padding = 'same', activation='relu', name = 'conv1')(x)
    x = tf.keras.layers.Conv1D(64, 1, padding = 'same', activation='relu', name = 'conv2')(x)
    x = tf.keras.layers.MaxPool1D(2)(x)
    x = tf.keras.layers.Conv1D(32, 3, padding = 'same', activation='relu', name = 'conv3')(x)
    x = tf.keras.layers.Conv1D(32, 1, padding = 'same', activation='relu', name = 'conv4')(x)
    x = tf.keras.layers.MaxPool1D(2)(x)
    x = tf.keras.layers.Flatten(name = 'flatten')(x)
    x = tf.keras.layers.Dense(64, activation='relu', name = 'relu')(x)
    pred = tf.keras.layers.Dense(2, activation='softmax', name = 'output')(x) 
    model = tf.keras.Model(inputs=[*inputs], outputs=pred)

    return model


In [ ]:
base_model = build_base_model()
base_model.summary()

### Train and evaluate the model on perturbed dataset

In [ ]:
base_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                   metrics=['acc'])

base_model.fit(perturbed_data_train_list[:-1], getLabels(perturbed_data_train_list), epochs=HPARAMS.epochs)


In [ ]:

results = base_model.evaluate(perturbed_data_test_list[:-1], getLabels(perturbed_data_test_list))
named_results = dict(zip(base_model.metrics_names, results))
print('\naccuracy:', named_results['acc'])

## Adversarial-regularized model


In [ ]:
base_adv_model = build_base_model()
adv_model = nsl.keras.AdversarialRegularization(
    base_adv_model,
    label_keys = ['labels']
)

In [ ]:
adv_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                   metrics=['acc'])


adv_model.fit(perturbed_data_train_dict, epochs=HPARAMS.epochs)

In [ ]:
results = adv_model.evaluate(perturbed_data_test_dict)
named_results = dict(zip(adv_model.metrics_names, results))
print('\naccuracy:', named_results['sparse_categorical_accuracy'])

## Evaluate models with un-perturbed datasets

In [ ]:
adv_config= nsl.configs.make_adv_reg_config(
    multiplier=HPARAMS.adv_multiplier,
    adv_step_size=HPARAMS.adv_step_size,
    adv_grad_norm=HPARAMS.adv_grad_norm
)

In [ ]:
reference_model = nsl.keras.AdversarialRegularization(
    base_model,
    label_keys = ['labels'],
    adv_config=adv_config
)
reference_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['acc'])

In [ ]:
models_to_eval = {
    'base': base_model,
    'adv-regularized': adv_model.base_model
}
metrics = {
    name: tf.keras.metrics.SparseCategoricalAccuracy()
    for name in models_to_eval.keys()
}

In [ ]:
pred_cut = 0.6
predictions = []
predictions.append({})


perturbed_batch = [pbatch[:200] for pbatch in perturbed_data_test_list[:-1]]

sample_batch = [sbatch[:200] for sbatch in data_test_list[:-1]]


y_true = np.array(perturbed_batch)
    
for name, model in models_to_eval.items():
    print('evaluating {0} model'.format(name))
    y_pred = model(data_test_dict)

    metrics[name](y_true, y_pred)
    
    y = np.where(y_pred[:,1] > pred_cut, 1, 0)
    predictions[-1][name] = y
    

In [ ]:
plt.rcParams['figure.dpi'] = 100

In [ ]:


batch_index = slice(0,200)
    
n_col = 3
n_row = 1

labels =  getLabels(data_test_list)

sig_true = (labels == 1)
bkg_true = (labels == 0)



sig_base = (predictions[0]['base'] == 1)
bkg_base = (predictions[0]['base'] == 0)

sig_adv = (predictions[0]['adv-regularized'] == 1)
bkg_adv = (predictions[0]['adv-regularized'] == 0)

for i, feature in enumerate(data_test_list):
    if(i==len(data_test_list)-1):continue
    

    fig, ax = plt.subplots(n_row, n_col, figsize=(12, 4) )
    bmax = np.max(feature.flatten())
    bmin = np.min(feature.flatten())
    
    ax[0].hist(feature[sig_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[0].hist(feature[bkg_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[0].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[0].set_ylabel('Counts')
    ax[0].set_title('Labels')
    ylim = ax[0].get_ylim()

    ax[1].hist(feature[sig_base].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[1].hist(feature[bkg_base].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[1].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[1].set_ylabel('Counts')
    ax[1].set_title('Base Model')
    ax[1].set_ylim(ylim)
    
    ax[2].hist(feature[sig_adv].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[2].hist(feature[bkg_adv].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[2].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[2].set_ylabel('Counts')
    ax[2].set_title('Adv Regularized Model')
    ax[2].set_ylim(ylim)
    plt.show()



### Correctly Predicted Events

In [ ]:
for i, feature in enumerate(data_test_list):
    if(i==len(data_test_list)-1):continue
    

    fig, ax = plt.subplots(n_row, n_col, figsize=(12, 4) )
    bmax = np.max(feature.flatten())
    bmin = np.min(feature.flatten())
    
    ax[0].hist(feature[sig_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[0].hist(feature[bkg_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[0].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[0].set_ylabel('Counts')
    ax[0].set_title('Labels')
    ylim = ax[0].get_ylim()

    ax[1].hist(feature[sig_base & sig_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[1].hist(feature[bkg_base & bkg_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[1].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[1].set_ylabel('Counts')
    ax[1].set_title('Base Model')
    ax[1].set_ylim(ylim)
    
    ax[2].hist(feature[sig_adv & sig_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[2].hist(feature[bkg_adv & bkg_true].flatten(), bins=np.linspace(bmin, bmax, 50), alpha=0.5)
    ax[2].set_xlabel(FEATURE_INPUT_NAME[i])
    ax[2].set_ylabel('Counts')
    ax[2].set_title('Adv Regularized Model')
    ax[2].set_ylim(ylim)
    plt.show()


In [ ]:
true_sig = len(sig_true[sig_true & sig_true])
true_bkg = len(bkg_true[bkg_true & bkg_true])

true_base_sig = len(sig_true[sig_true & sig_base])
true_base_bkg = len(bkg_true[bkg_true & bkg_base])

true_adv_sig = len(sig_true[sig_true & sig_adv])
true_adv_bkg = len(bkg_true[bkg_true & bkg_adv])

print()
print('Total Labels', true_sig + true_bkg)
print('Total Correctly Predicted Base Model', true_base_sig + true_base_bkg)
print('Total Correctly Predicted Adv Model ', true_adv_sig + true_adv_bkg)
print()
print('True Signal', true_sig)
print('Correctly Predicted Signal, Base Model', true_base_sig)
print('Correctly Predicted Signal, Adv Model ', true_adv_sig)
print()
print('True Background', true_bkg)
print('Correctly Predicted Background, Base Model', true_base_bkg)
print('Correctly Predicted Background, Adv Model ', true_adv_bkg)


